In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import lightgbm as lgb

from hyperopt import hp, tpe, fmin, Trials
from hyperopt.pyll.base import scope

import mlflow
import mlflow.lightgbm

import sys
import os
sys.path.append('/Users/ben/Desktop/py_proj/account_funds_prediction')
sys.path.append('/Users/ben/Desktop/py_proj/account_funds_prediction/src')
sys.path.append('/Users/ben/Desktop/py_proj/account_funds_prediction/utils')

#from src.lightgbm_model_v3 import lightgbm_dev
from src.lightgbm_model import *
from utils.metrics import *
from utils.plots import *

import warnings
warnings.filterwarnings("ignore")

In [2]:
sys.path

['/Library/Frameworks/Python.framework/Versions/3.9/lib/python39.zip',
 '/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9',
 '/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/lib-dynload',
 '',
 '/Users/ben/Desktop/py_proj/account_funds_prediction/venv1/lib/python3.9/site-packages',
 '/Users/ben/Desktop/py_proj/account_funds_prediction',
 '/Users/ben/Desktop/py_proj/account_funds_prediction/src',
 '/Users/ben/Desktop/py_proj/account_funds_prediction/utils']

In [3]:
os.getcwd()

'/Users/ben/Desktop/py_proj/account_funds_prediction/notebook'

## 1. Data prep

In [4]:
df = pd.read_csv('/Users/ben/Desktop/py_proj/account_funds_prediction/data/df.csv')

In [5]:
df.head()

,id,funds_now,funds_after_6months,age,tenure,credit_score,annual_income,mtg_balance,credit_card_balance,loan_balance,...,fea41,fea42,fea43,fea44,fea45,fea46,fea47,fea48,fea49,fea50
0,1,580888.6,411571.0,69,44,443,158191,211023.6,13320.2,11516.5,...,10232.7,26721.0,39983.1,45532.4,11885.7,1,1,1,0,1
1,2,372438.7,157944.4,32,43,496,208050,19841.3,45983.3,119858.9,...,-26726.7,-19164.4,22911.3,-9997.3,3621.1,1,0,0,0,0
2,3,645639.9,22235.5,89,55,527,148731,854416.8,41016.9,31300.6,...,-15244.7,4683.6,32100.5,-32710.4,39201.2,1,1,1,1,0
3,4,1191515.2,316512.4,78,1,628,263843,850936.3,2612.9,73886.1,...,18567.4,28572.1,-44307.5,-43039.7,156.0,1,1,1,1,0
4,5,348260.8,0.0,38,16,764,264430,979270.8,5583.3,252329.3,...,-15909.1,-33315.5,-32351.8,15200.4,-7349.3,0,0,1,1,0


In [6]:
X = df.drop(columns=['funds_after_6months', 'id'])
y = df['funds_after_6months']

In [7]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

## 2. Model Training

### 2.1 Baseline model

In [8]:
baseline_params = {
    'max_depth': 3,
    'n_estimators': 50,
    'learning_rate': 0.1,
    'feature_fraction': 0.5
}

search_space = {
    # Integer parameters
    'max_depth': scope.int(hp.quniform('max_depth', 3,10,1)),
    'n_estimators': scope.int(hp.quniform('n_estimators', 50,300,50)),
    'num_leaves': scope.int(hp.quniform('num_leaves', 20, 100, 5)),  
    'min_data_in_leaf': scope.int(hp.quniform('min_data_in_leaf', 5, 100, 5)),  

    # Decimal parameters
    'learning_rate': hp.uniform('learning_rate', 0.01, 1),  # Typical range for learning rate
    'feature_fraction': hp.uniform('feature_fraction', 0.3, 1.0),  # Typical range for feature_fraction
    'bagging_fraction': hp.uniform('bagging_fraction', 0.3, 1.0),  # Typical range for bagging_fraction
    'lambda_l1': hp.uniform('lambda_l1', 0, 1),  # Typical range for lambda_l1 (L1 regularization)
}


In [9]:
lgb_dev1 = lightgbm_dev_v2(X_train=X_train, y_train=y_train, 
                   X_test=X_test, y_test=y_test,
                   baseline_params = baseline_params,
                   baseline_ind=1,
                   max_evals=2,
                   search_space=search_space)

In [10]:
lgb_dev1

In [11]:
lgb1 = lgb_dev1.run_model()

[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002305 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10335
[LightGBM] [Info] Number of data points in the train set: 140000, number of used features: 50
[LightGBM] [Info] Start training from score 177372.881641
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5


2024/12/29 15:18:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


MLflow Run ID: 968508395d6e46c2a5732d119258e1d8


In [12]:
lgb1

(LGBMRegressor(feature_fraction=0.5, max_depth=3, n_estimators=50),
 '968508395d6e46c2a5732d119258e1d8')

In [13]:
lgb1_id = lgb1[1]
lgb1_id

'968508395d6e46c2a5732d119258e1d8'

In [14]:
# Fetch the run details using MLflow API
run = mlflow.get_run(lgb1_id)

# Retrieve the metrics from the run
rmse_train = run.data.metrics.get("rmse_train", None)
rmse_test = run.data.metrics.get("rmse_test", None)
r2_train = run.data.metrics.get("r2_train", None)
r2_test = run.data.metrics.get("r2_test", None)
pr_train = run.data.metrics.get("powerratio_train", None)
pr_test = run.data.metrics.get("powerratio_test", None)

# Print the retrieved metrics
print(f"RMSE Train: {rmse_train}")
print(f"RMSE Test: {rmse_test}")
print(f"R2 Train: {r2_train}")
print(f"R2 Test: {r2_test}")
print(f"Power Ratio Train: {pr_train}")
print(f"Power Ratio Test: {pr_test}")

RMSE Train: 89797.37195508531
RMSE Test: 88598.92600274534
R2 Train: 0.8255338855969169
R2 Test: 0.8293940009814278
Power Ratio Train: 0.9398507959736816
Power Ratio Test: 0.9390895402733672


### 2.2 HyperOpt (32 fits)

In [15]:
lgb_dev2 = lightgbm_dev_v2(X_train=X_train, y_train=y_train, 
                   X_test=X_test, y_test=y_test,
                   baseline_params = baseline_params,
                   baseline_ind=0,
                   max_evals=32,
                   search_space=search_space)

In [16]:
%%time
lgb2 = lgb_dev2.run_model()

[LightGBM] [Warning] lambda_l1 is set=0.012726268418242204, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.012726268418242204                                        
[LightGBM] [Warning] feature_fraction is set=0.3676942464853764, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3676942464853764                       
[LightGBM] [Warning] bagging_fraction is set=0.872318944631925, subsample=1.0 will be ignored. Current value: bagging_fraction=0.872318944631925                                
[LightGBM] [Warning] min_data_in_leaf is set=40, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=40                                                       
[LightGBM] [Warning] lambda_l1 is set=0.012726268418242204, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.012726268418242204                                        
[LightGBM] [Warning] feature_fraction is set=0.3676942464853764, colsample_bytree=1.0 will be ignored. Current valu

2024/12/29 15:20:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


MLflow Run ID: 6a2b05589a9a4118852e876c57b64d92
best param: {'num_leaves': 90, 'max_depth': 3, 'n_estimators': 250, 'min_data_in_leaf': 70, 'learning_rate': 0.10339912143177958, 'feature_fraction': 0.8952695235095387, 'bagging_fraction': 0.9892612556006475, 'lambda_l1': 0.13321591848299014}
CPU times: user 6min 43s, sys: 1min 56s, total: 8min 40s
Wall time: 1min 26s


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                                                                          
[LightGBM]

2024/12/26 13:49:03 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during lightgbm autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 7acde082c46148dbb0f568f9e9350173. Failed operations: [MlflowException("Changing param values is not allowed. Param with key=\'learning_rate\' was already logged with value=\'0.730236549985059\' for run ID=\'7acde082c46148dbb0f568f9e9350173\'. Attempted logging new value \'0.8769880053165809\'.")]')]



[LightGBM] [Warning] lambda_l1 is set=0.2570035237866054, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.2570035237866054                                
[LightGBM] [Warning] feature_fraction is set=0.4460491275969355, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4460491275969355           
[LightGBM] [Warning] bagging_fraction is set=0.8704779571454411, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8704779571454411                  
[LightGBM] [Warning] min_data_in_leaf is set=95, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=95                                           
[LightGBM] [Warning] lambda_l1 is set=0.2570035237866054, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.2570035237866054                                
[LightGBM] [Warning] feature_fraction is set=0.4460491275969355, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4460491275969355           
[LightGBM]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [17]:
lgb2

(LGBMRegressor(bagging_fraction=0.9892612556006475,
               feature_fraction=0.8952695235095387,
               lambda_l1=0.13321591848299014, learning_rate=0.10339912143177958,
               max_depth=3, min_data_in_leaf=70, n_estimators=250,
               num_leaves=90),
 '6a2b05589a9a4118852e876c57b64d92')

In [18]:
# Fetch the run details using MLflow API
run_id = lgb2[1]
run = mlflow.get_run(run_id)

# Retrieve the metrics from the run
rmse_train = run.data.metrics.get("rmse_train", None)
rmse_test = run.data.metrics.get("rmse_test", None)
r2_train = run.data.metrics.get("r2_train", None)
r2_test = run.data.metrics.get("r2_test", None)
pr_train = run.data.metrics.get("powerratio_train", None)
pr_test = run.data.metrics.get("powerratio_test", None)

# Print the retrieved metrics
print(f"RMSE Train: {rmse_train}")
print(f"RMSE Test: {rmse_test}")
print(f"R2 Train: {r2_train}")
print(f"R2 Test: {r2_test}")
print(f"Power Ratio Train: {pr_train}")
print(f"Power Ratio Test: {pr_test}")

RMSE Train: 85295.04435736763
RMSE Test: 85244.78179371079
R2 Train: 0.8425903205994085
R2 Test: 0.8420669617503107
Power Ratio Train: 0.9421585815197544
Power Ratio Test: 0.9404864764481451
